In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from scipy.spatial.distance import pdist, squareform
from datetime import datetime
from ceto.analysis import frechet_distance, douglas_peucker
import folium
import random


In [12]:


# Segment data into trajectories
def segment_data(data, stop_threshold, speed_threshold = 0.5):
    trajectories = []
    stops = []
    current_trajectory = []
    current_stop = []
    aggregated_movement = 0

    for index, (timestamp, lat, lon, sog, fc_me, fc_ae) in enumerate(data):
        current_time = timestamp
        if index > 0:
            time_diff = (current_time - previous_time).seconds
            aggregated_movement += sog * time_diff

        if sog > speed_threshold and aggregated_movement >= stop_threshold:
            if current_stop:
                stops.append(current_stop)
                current_stop = []
            current_trajectory.append((timestamp, lat, lon, sog, fc_me, fc_ae))
            aggregated_movement = 0
        else:
            if current_trajectory:
                trajectories.append(current_trajectory)
                current_trajectory = []
            current_stop.append((timestamp, lat, lon, sog, fc_me, fc_ae))

        previous_time = current_time

    if current_trajectory:
        trajectories.append(current_trajectory)
    elif current_stop:
        stops.append(current_stop)

    return trajectories, stops

def get_path(trajectory):
    """
    trajectory: (timestamp, latitude, longitude, xxx, yyy, zzz)
    """
    return [(point[1], point[2]) for point in trajectory]



# Cluster trajectories using DBSCAN
def cluster_trajectories(trajectories, eps, min_samples=2, epsilon=10):
    """
    eps: maximum frechet distance for clustering
    epsilon: simplification distance

    """
    paths = [douglas_peucker(get_path(traj), epsilon) for traj in trajectories]
    distance_matrix = squareform(pdist(paths, metric=frechet_distance(x, y)))
    clustering = DBSCAN(eps=eps, min_samples=min_samples, metric='precomputed')
    labels = clustering.fit_predict(distance_matrix)

    return labels

# Generate representative trajectory for each cluster
def representative_trajectories(trajectories, labels):
    n_clusters = len(set(labels))
    cluster_centers = []

    for cluster_idx in range(n_clusters):
        cluster_points = [traj for traj, label in zip(trajectories, labels) if label == cluster_idx]
        mean_trajectory_length = int(np.mean([len(traj) for traj in cluster_points]))

        # Resample trajectories in the cluster to have the same length
        resampled_trajectories = []
        for traj in cluster_points:
            resampled_trajectory = np.interp(np.linspace(0, len(traj), mean_trajectory_length), np.arange(len(traj)), traj)
            resampled_trajectories.append(resampled_trajectory)

        cluster_center = np.mean(resampled_trajectories, axis=0)
        cluster_centers.append(cluster_center)

    return cluster_centers

# Generate a histogram of the total fuel consumption for each trajectory cluster
def fuel_consumption_histogram(trajectories, labels):
    n_clusters = len(set(labels))
    fuel_consumption = [0] * n_clusters

    for traj, label in zip(trajectories, labels):
        fuel_consumption[label] += sum(point[2] for point in traj)

    # Plot the histogram
    plt.bar(range(n_clusters), fuel_consumption)
    plt.xlabel('Cluster')
    plt.ylabel('Total Fuel Consumption')
    plt.title('Fuel Consumption per Trajectory Cluster')
    plt.show()

COLORS = [
    "#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b",
    "#e377c2", "#7f7f7f", "#bcbd22", "#17becf", "#e6194b", "#3cb44b",
    "#f58231", "#911eb4", "#46f0f0", "#f032e6", "#bcf60c", "#fabebe",
    "#008080", "#e6beff", "#9a6324", "#fffac8", "#800000", "#aaffc3",
    "#808000", "#ffd8b1", "#000075", "#a9a9a9", "#ffffff", "#000000"
]

def plot_trajectories(trajectories, epsilon=10, zoom_start=10, labels=None, simplify=True):

    if simplify:
        paths = [douglas_peucker(get_path(traj), epsilon) for traj in trajectories]
    else:
        paths = [get_path(traj) for traj in trajectories]

    # Find the map center
    all_points = [point for path in paths for point in path]
    avg_lat = sum(point[0] for point in all_points) / len(all_points)
    avg_lon = sum(point[1] for point in all_points) / len(all_points)

    # Create a map centered at the average latitude and longitude
    map = folium.Map(location=[avg_lat, avg_lon], zoom_start=zoom_start)

    # Plot paths
    for i, path in enumerate(paths):
        start_point = path[0]
        end_point = path[-1]
        
        if labels is not None:
            label = labels[i]
            if label == -1:
                color = 'black'
            else:
                color = COLORS[labels[i]]
            popup_text = f"Start: {start_point}\nEnd: {end_point}\nLabel: {label}"
        else:
            color = "blue"
            popup_text = f"Start: {start_point}\nEnd: {end_point}\n"
        polyline = folium.PolyLine(path, color=color, weight=2.5, opacity=1)
        
        # Add a popup to display start point, end point, and label when clicked
        polyline_popup = folium.Popup(popup_text, max_width=300)
        polyline.add_child(polyline_popup)
        polyline.add_to(map)

    return map






In [29]:
# Parameters
file_path = 'data/2022-11-20 - Djurga╠èrden 11 2 dagar mer vind.csv'
stop_duration = 1  # minutes 
speed_threshold = 1 # knots
epsilon = 5  # Tolerance for the Douglas-Peucker algorithm
n_clusters = 4  # Number of clusters

In [30]:
# Load data and split into trajectories
df = pd.read_csv(file_path)

In [31]:
# Change column names
df.rename(columns = {
    "Timestamp [UTC]":"ts",
    "Latitude (deg)": "lat",
    "Longitude (deg)": "lon",
    "Speed over ground (kts)":"sog",
    "Consumption ME (L/h)": "fc_me",
    "Consumption AE (L/h)": "fc_ae"
}, inplace = True)

# Timestamp string to datetime object
df["ts"] = pd.to_datetime(df["ts"])

# Change column order
df = df[['ts','lat','lon','sog','fc_me','fc_ae']]

# Drop timestamp duplicates
df = df.drop_duplicates(subset='ts',keep='first') \
    .sort_values(by='ts')

# Drop data points where the speed is below 0.5 kn
df = df.drop(df[df['sog'] < speed_threshold].index)

# Add column with time between messages (dt)
df['dt'] = pd.Series(
    np.diff(df.ts).astype('timedelta64[m]'),
    index=df.index[1:]
    )

# Split data into trajectories at time gaps ( dt > 2 min )
trajectories = []
for group in np.split(df, np.where(df.dt > pd.Timedelta(f'{stop_duration} min'))[0][1:]):
    trajectories.append(list(group.to_records(index=False)))

In [32]:
# Get paths from trajectories
map = plot_trajectories(trajectories, zoom_start=15, epsilon=epsilon)
map

In [33]:
paths = [douglas_peucker(get_path(traj), epsilon) for traj in trajectories]
distance_matrix = np.zeros([len(paths),len(paths)])
for i, i_path in enumerate(paths):
    for j, j_path in enumerate(paths):
        distance_matrix[i,j] = frechet_distance(i_path, j_path)
clustering = DBSCAN(eps=100, min_samples=3, metric='precomputed')
labels = clustering.fit_predict(distance_matrix)

plot_trajectories(trajectories,labels=labels,epsilon=epsilon,simplify=True, zoom_start=15)

In [24]:
distance_matrix

array([[   0.        , 1016.9698294 ,  742.45542841, ...,  750.54916618,
        1013.53208731,  932.46113695],
       [1016.9698294 ,    0.        , 1021.04172239, ..., 1019.29233126,
          74.76136507, 1019.73546861],
       [ 742.45542841, 1021.04172239,    0.        , ...,  133.78889349,
        1021.60531002,  937.61818706],
       ...,
       [ 750.54916618, 1019.29233126,  133.78889349, ...,    0.        ,
        1019.85625365,  934.49390131],
       [1013.53208731,   74.76136507, 1021.60531002, ..., 1019.85625365,
           0.        , 1020.29946134],
       [ 932.46113695, 1019.73546861,  937.61818706, ...,  934.49390131,
        1020.29946134,    0.        ]])

In [ ]:
# Draw trajectories from time-grouped messages and segment them
# by basic behaviours
for group in grouped_messages:
    subset = group[['lon', 'lat', 'timestamp']]
    trajectory = [tuple(x) for x in subset.

In [9]:
df.rename(columns = {
    "Timestamp [UTC]":"timestamp",
    "Speed over ground (kts)":"sog_kts",
    "Consumption ME (L/h)": "fuel_consumption_main_engines",
    "Consumption AE (L/h)": "fuel_consumption_auxiliary_engines",
    "Latitude (deg)": "latitude",
    "Longitude (deg)": "longitude"
}, inplace = True)

In [55]:
trajectories[-1][0]

(Timestamp('2022-11-21 18:00:07+0000', tz='UTC'), 59.32297, 18.07639, 1.01, 42.3, 2.99999, 240000000000)

In [56]:
# Remove 
trajectories[-1][-1]

(Timestamp('2022-11-21 18:11:22+0000', tz='UTC'), 59.32863, 18.08058, 1.26, 27.7, 2.99999, 0)

In [57]:
trajectories[-1]

[(Timestamp('2022-11-21 18:00:07+0000', tz='UTC'), 59.32297, 18.07639, 1.01, 42.3, 2.99999, 240000000000),
 (Timestamp('2022-11-21 18:00:08+0000', tz='UTC'), 59.32297, 18.0764, 1.37, 42.1, 2.99999, 0),
 (Timestamp('2022-11-21 18:00:09+0000', tz='UTC'), 59.32297, 18.07642, 1.77, 41.3, 2.99999, 0),
 (Timestamp('2022-11-21 18:00:10+0000', tz='UTC'), 59.32297, 18.07644, 2.17, 40.7, 2.99999, 0),
 (Timestamp('2022-11-21 18:00:11+0000', tz='UTC'), 59.32296, 18.07645, 2.62, 40., 2.99999, 0),
 (Timestamp('2022-11-21 18:00:12+0000', tz='UTC'), 59.32296, 18.07648, 2.89, 39.7, 2.99999, 0),
 (Timestamp('2022-11-21 18:00:13+0000', tz='UTC'), 59.32296, 18.07651, 3.3, 39.4, 2.99999, 0),
 (Timestamp('2022-11-21 18:00:14+0000', tz='UTC'), 59.32295, 18.07653, 3.56, 39.1, 2.99999, 0),
 (Timestamp('2022-11-21 18:00:15+0000', tz='UTC'), 59.32295, 18.07657, 3.77, 39.1, 2.99999, 0),
 (Timestamp('2022-11-21 18:00:16+0000', tz='UTC'), 59.32294, 18.0766, 4.13, 38.8, 2.99999, 0),
 (Timestamp('2022-11-21 18:00:17+